In [1]:
import pandas as pd
#import pandavro as pda
import lxml.html as lh
from urllib.request import urlopen, Request
import requests
pd.__version__


'1.0.1'

In [2]:
url = "http://www.brewunited.com/grain_database.php"
print(url)

http://www.brewunited.com/grain_database.php


In [3]:


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(url=url, headers=headers) 
page = requests.get(url)

print(page) 
doc = lh.fromstring(page.content)

tr_elements = doc.xpath('//tr')
print(tr_elements)


#Check the length of the first 50 rows
[len(T) for T in tr_elements[:50]]


<Response [200]>
[<Element tr at 0x292befe9188>, <Element tr at 0x292befe94a8>, <Element tr at 0x292befe9458>, <Element tr at 0x292befe94f8>, <Element tr at 0x292befe9548>, <Element tr at 0x292befe9598>, <Element tr at 0x292befe95e8>, <Element tr at 0x292befe9638>, <Element tr at 0x292befe9688>, <Element tr at 0x292befe96d8>, <Element tr at 0x292befe9728>, <Element tr at 0x292befe9778>, <Element tr at 0x292befe97c8>, <Element tr at 0x292befe9818>, <Element tr at 0x292befe9868>, <Element tr at 0x292befe98b8>, <Element tr at 0x292befe9908>, <Element tr at 0x292befe9958>, <Element tr at 0x292befe99a8>, <Element tr at 0x292befe99f8>, <Element tr at 0x292befe9a48>, <Element tr at 0x292befe9a98>, <Element tr at 0x292befe9ae8>, <Element tr at 0x292befe9b38>, <Element tr at 0x292befe9b88>, <Element tr at 0x292befe9bd8>, <Element tr at 0x292befe9c28>, <Element tr at 0x292befe9c78>, <Element tr at 0x292befe9cc8>, <Element tr at 0x292befe9d18>, <Element tr at 0x292befe9d68>, <Element tr at 0x292b

[8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8]

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"' %(i,name))
    col.append((name,[]))

1:"Grain"
2:"Origin"
3:"Mash?"
4:"Color"
5:"Power"
6:"Potential"
7:"Max %"
8:"Notes"


In [5]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=8:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head(20) 

,Grain,Origin,Mash?,Color,Power,Potential,Max %,Notes
0,Acid Malt,Germany,X,3,0,1.027,10,Acid malt contains acids from natural lactic a...
1,Acidulated (Weyermann),Germany,,1.8,0,1.030,10,Used in Germany to lower PH levels without res...
2,Acidulated Malt,German,,3.4,0,0.000,0,
3,Amber Malt,United Kingdom,X,22,20,1.035,20,Roasted specialty malt used in some English br...
4,Aromatic Malt,Belgium,X,26,29,1.036,10,Provides a very strong malt flavor and aroma t...
5,"Barley, Flaked",US,X,1.7,0,1.032,20,Adds significant body to Porters and Stouts. H...
6,"Barley, Raw",US,X,2,0,1.028,15,"Raw, unmalted barley can be used to add body t..."
7,"Barley, Roasted",US,,300,0,1.025,10,"Roasted at high temperature to create a burnt,..."
8,"Barley, Torrefied",US,X,1.7,0,1.036,40,"Raw barley that has been ""popped"" open to open..."
9,Belgian Debittered Black Malt,,,550,0,1.010,0,


In [8]:
df.count()

Grain        258
Origin       258
Mash?        258
Color        258
Power        258
Potential    258
Max %        258
Notes        258
dtype: int64

In [11]:
##cleaning Duplicates and rubish
df = df.drop_duplicates()
df.count()



#df = df[df['Hop'] != 'Hop']
#df = df[df['Hop'] != '0']

Grain        183
Origin       183
Mash?        183
Color        183
Power        183
Potential    183
Max %        183
Notes        183
dtype: int64

In [12]:
df.head(60)

,Grain,Origin,Mash?,Color,Power,Potential,Max %,Notes
0,Acid Malt,Germany,X,3,0,1.027,10,Acid malt contains acids from natural lactic a...
1,Acidulated (Weyermann),Germany,,1.8,0,1.030,10,Used in Germany to lower PH levels without res...
2,Acidulated Malt,German,,3.4,0,0.000,0,
3,Amber Malt,United Kingdom,X,22,20,1.035,20,Roasted specialty malt used in some English br...
4,Aromatic Malt,Belgium,X,26,29,1.036,10,Provides a very strong malt flavor and aroma t...
5,"Barley, Flaked",US,X,1.7,0,1.032,20,Adds significant body to Porters and Stouts. H...
6,"Barley, Raw",US,X,2,0,1.028,15,"Raw, unmalted barley can be used to add body t..."
7,"Barley, Roasted",US,,300,0,1.025,10,"Roasted at high temperature to create a burnt,..."
8,"Barley, Torrefied",US,X,1.7,0,1.036,40,"Raw barley that has been ""popped"" open to open..."
9,Belgian Debittered Black Malt,,,550,0,1.010,0,


In [13]:

df = df[df['Origin'] != 'Origin']

df.count()

Grain        182
Origin       182
Mash?        182
Color        182
Power        182
Potential    182
Max %        182
Notes        182
dtype: int64

In [14]:
df.head(60)

,Grain,Origin,Mash?,Color,Power,Potential,Max %,Notes
0,Acid Malt,Germany,X,3,0,1.027,10,Acid malt contains acids from natural lactic a...
1,Acidulated (Weyermann),Germany,,1.8,0,1.030,10,Used in Germany to lower PH levels without res...
2,Acidulated Malt,German,,3.4,0,0.000,0,
3,Amber Malt,United Kingdom,X,22,20,1.035,20,Roasted specialty malt used in some English br...
4,Aromatic Malt,Belgium,X,26,29,1.036,10,Provides a very strong malt flavor and aroma t...
5,"Barley, Flaked",US,X,1.7,0,1.032,20,Adds significant body to Porters and Stouts. H...
6,"Barley, Raw",US,X,2,0,1.028,15,"Raw, unmalted barley can be used to add body t..."
7,"Barley, Roasted",US,,300,0,1.025,10,"Roasted at high temperature to create a burnt,..."
8,"Barley, Torrefied",US,X,1.7,0,1.036,40,"Raw barley that has been ""popped"" open to open..."
9,Belgian Debittered Black Malt,,,550,0,1.010,0,


In [110]:
df.to_json (r'C:\Users\bceolincamar\Documents\GitHub\homebrew\Export_Grains.json', orient='records')

In [16]:
df.rename(columns={ df.columns[6]: "MaxPercent" }, inplace = True)
df.rename(columns={ df.columns[2]: "Mash" }, inplace = True)

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 0 to 249
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Grain       168 non-null    object
 1   Origin      168 non-null    object
 2   Mash        168 non-null    object
 3   Color       168 non-null    object
 4   Power       168 non-null    object
 5   Potential   168 non-null    object
 6   MaxPercent  168 non-null    object
 7   Notes       168 non-null    object
dtypes: object(8)
memory usage: 11.8+ KB


In [19]:
df = df.sort_values(by=['Grain'])

In [22]:
df.head(60)

,Grain,Origin,Mash,Color,Power,Potential,MaxPercent,Notes
0,Acid Malt,Germany,X,3,0,1.027,10,Acid malt contains acids from natural lactic a...
1,Acidulated (Weyermann),Germany,,1.8,0,1.030,10,Used in Germany to lower PH levels without res...
2,Acidulated Malt,German,,3.4,0,0.000,0,
3,Amber Malt,United Kingdom,X,22,20,1.035,20,Roasted specialty malt used in some English br...
4,Aromatic Malt,Belgium,X,26,29,1.036,10,Provides a very strong malt flavor and aroma t...
5,"Barley, Flaked",US,X,1.7,0,1.032,20,Adds significant body to Porters and Stouts. H...
6,"Barley, Raw",US,X,2,0,1.028,15,"Raw, unmalted barley can be used to add body t..."
7,"Barley, Roasted",US,,300,0,1.025,10,"Roasted at high temperature to create a burnt,..."
8,"Barley, Torrefied",US,X,1.7,0,1.036,40,"Raw barley that has been ""popped"" open to open..."
9,Belgian Debittered Black Malt,,,550,0,1.010,0,


In [121]:
df['Grain'].value_counts()
 

dp.head(30)

Crystal 60, 2-Row, (Great Western)    1
Blackprinz Malt (Briess)              1
Oats, Flaked (toasted)                1
Wheat Malt, Bel                       1
Chocolate Malt                        1
RedX (BestMälz)                       1
Corn - Yellow, Flaked (Briess)        1
Munich Malt                           1
Pale Malt (2 Row), NW Pale            1
Munich Malt - 10L                     1
Pale Malt (2 Row) UK                  1
Special Roast                         1
Crystal 40, 2-Row, (Great Western)    1
Wheat, Torrified                      1
Chocolate Malt (Muntons)              1
Rice, Flaked (Briess)                 1
Vienna (BestMälz)                     1
Home Roasted Pilsner                  1
Pilsner (2 Row) UK                    1
Wheat, Flaked                         1
Special B Malt                        1
Caramel/Crystal Malt - 20L            1
Wheat - White Malt (Briess)           1
Carafa Special II (Weyermann)         1
White Wheat Malt                      1


AttributeError: 'Series' object has no attribute 'merge'

In [43]:
dfn = df.groupby(
   ['Grain']
).agg(
    {
         
         'Grain': "count"  # get the count of networks
         
    }
)
dfn.head(60)
dfn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 168 entries, Acid Malt to White Wheat Malt
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Grain   168 non-null    int64
dtypes: int64(1)
memory usage: 2.6+ KB


In [49]:
df.groupby('Grain').filter(lambda group: len(group) > 1).sort('Grain', ascending=False)


AttributeError: 'DataFrame' object has no attribute 'sort'

In [107]:
dup = df[df.duplicated(['Grain'])].sort_values(by=['Grain'])

In [108]:
dup.head(50)

,Grain,Origin,Mash,Color,Power,Potential,MaxPercent,Notes


In [103]:
df[df['Grain'] == 'Munich I (Weyermann)']

,Grain,Origin,Mash,Color,Power,Potential,MaxPercent,Notes
130,Munich I (Weyermann),Germany,,7,5,1.000,1,Light Munich malt. May be used as a base for m...
129,Munich I (Weyermann),Germany,,7.1,50,1.038,100,Light Munich malt. May be used as a base for m...


In [104]:
#dfne = df[df['Grain'] != 'Caramunich I (Weyermann)'] and df[df['Color'] != '34.5']
df = df.drop([129])

In [105]:
df[df['Grain'] == 'Caramunich I (Weyermann)']

,Grain,Origin,Mash,Color,Power,Potential,MaxPercent,Notes
54,Caramunich I (Weyermann),Germany,,51,0,1.036,10,"German crystal malt. Adds maltiness, flavor, c..."


In [106]:
df.count()

Grain         168
Origin        168
Mash          168
Color         168
Power         168
Potential     168
MaxPercent    168
Notes         168
dtype: int64